In [5]:
import msprime, pyslim
import tskit
import json
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import itertools
import math

import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import os
import pandas as pd
from torch.utils.data import DataLoader, random_split, TensorDataset
from torch import nn

import sklearn
import sklearn.model_selection

rng = np.random.default_rng()

In [2]:
class KinDataset(Dataset):
    def __init__(self, annotations_file):
        self.labels = pd.read_csv(annotations_file, names = ["path", "N"])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input_matrix = pd.read_csv(self.labels.iloc[idx, 0], header = 0)
        input_tensor = torch.from_numpy(input_matrix.to_numpy().flatten()).float()
        label = self.labels.iloc[idx, 1]
        return input_tensor, label
    
kin_dataset = KinDataset("spatial_kin/labels.csv")
train_kin, test_kin = random_split(kin_dataset, [12, 12], generator=torch.Generator().manual_seed(42))
train_kin_dl = DataLoader(train_kin)
test_kin_dl = DataLoader(test_kin)
print(train_kin_dl)
print(test_kin_dl)

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [6]:
X_train = np.arange(10, dtype='float32').reshape((10, 1))
y_train = np.array([1.0, 1.3, 3.1, 2.0, 5.0, 6.3, 6.6, 
                    7.4, 8.0, 9.0], dtype='float32')
X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
X_train_norm = torch.from_numpy(X_train_norm)

# On some computers the explicit cast to .float() is
# necessary
y_train = torch.from_numpy(y_train).float()

train_ds = TensorDataset(X_train_norm, y_train)

batch_size = 1
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [ ]:
input_size = 1
output_size = 1

model = nn.Linear(input_size, output_size)

loss_fn = nn.MSELoss(reduction='mean')

learning_rate = 0.001
num_epochs = 200
log_epochs = 10

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for x_batch, y_batch in train_dl:
        # 1. Generate predictions
        pred = model(x_batch[0])

        # 2. Calculate loss
        loss = loss_fn(pred, y_batch)

        # 3. Compute gradients
        loss.backward()

        # 4. Update parameters using gradients
        optimizer.step()

        # 5. Reset the gradients to zero
        optimizer.zero_grad()
        
    if epoch % log_epochs==0:
        print(f'Epoch {epoch}  Loss {loss.item():.4f}')